If you have changed the default name, make sure you change the defaultLakehouse name as well

Make sure you have:
Create the Lakehouse LH_CONFIGURATION
Uploaded the 'FMD_deployment.json' json file to 'deployment' subfolder


In [200]:
%%configure -f

{ 
    "defaultLakehouse": { 
        "name":  "LH_CONFIGURATION"
    }
}

StatementMeta(, 5bcbb833-405a-41d2-a0ff-8103d99b7a53, -1, Finished, Available, Finished)

Before you runbook please change all the id's below. 

In [296]:
# source file to read the deployment manifest from (LH_CONFIGURATION) 
deployment_file = 'deployment/FMD_deployment.json'

# Before you run this Notebook change the id'd below
capacity_id = '075e5656-1234-5678-a36c8c0e8bca'

#Fabric SQL Framework
workspace_sql = '7774469b-1234-5678-bc42-a36c8c0e8bca'
artifact_sql_framework = '075e5656-1234-5678-89ed-63f6837c0ff6'

A#Add roles to the workpspace or use workspace_roles[] to add them in a later stage
workspace_roles = [
                    {
                        "principal": {
                            "id": "00000000-0000-0000-0000-000000000000",
                            "displayName": "sg-fabric-contributor",
                            "type": "Group"
                        },
                        "role": "Member"
                        },
                        {
                        "principal": {
                            "id": "00000000-0000-0000-0000-000000000000",
                            "displayName": "sg-fabric-admin",
                            "type": "Group"
                        },
                        "role": "Admin"
                        }
                    ]

logging = {
    "log_workspace_guid": "7db968e8-e86b-488e-b5ac-ffca009c89d7",
    "log_warehouse_guid": "bebda8eb-0b5f-41e9-933b-1cfc1df648f4",
    "log_warehouse_endpoint": "nl7yhqnbrscude3yv6mas6bxpq-5buls7ll5chernnm77fabhej24.datawarehouse.fabric.microsoft.com"
}

configuration = {
                    'workspace': {
                        'name' : 'FMD_FRAMEWORK_DATA_MH2 (dvlm)',
                        'roles' : workspace_roles,
                        'capacity_id' : capacity_id
                    },
}

environments = [
                    {
                        'environment_name' : 'development',
                        'workspaces': {
                            'data' : {
                                'name' : 'FMD_FRAMEWORK_DATA (D)',
                                'roles' : workspace_roles,
                                'capacity_id' : capacity_id
                            },
                            'code' : {
                                'name' : 'FMD_FRAMEWORK_CODE_MH2 (D)',
                                'roles' : workspace_roles,
                                'capacity_id' : capacity_id
                            }
                        },
                        'connections' : {
                            'CON_FMD_FSQL' : '372237f9-709a-48f8-8fb2-ce06940c990e',
                            'CON_FMD_API_FABRIC' : '644f217a-8761-4f19-b1b7-a40bfe0ec2af',
                            'CON_FMD_ASQL_01' : 'cf673e6a-13f6-4ebb-9cbb-4ba4ab390818',
                            'CON_FMD_ASQL_02' : '11a8e5fe-fbca-4822-9ba4-9162cf56e6dd',
                            'CON_FMD_ADLS_01' : 'a0581b6e-5e38-46eb-bab2-7f08e9a35c30',
                            'CON_FMD_FABRICPIPELINES' : '6d8146c6-a438-47df-94e2-540c552eb6d7',
                            'CON_FMD_ADF_PIPELINES' : '02e107b8-e97e-4b00-a28c-668cf9ce3d9a'
                        }
                    },
                    {
                        'environment_name' : 'production',
                        'workspaces': {
                            'data' : {
                                'name' : 'FMD_FRAMEWORK_DATA',
                                'roles' : workspace_roles,
                                'capacity_id' : capacity_id
                            },
                            'code' : {
                                'name' : 'FMD_FRAMEWORK_CODE_MH (prod)',
                                'roles' : workspace_roles,
                                'capacity_id' : capacity_id
                            }
                        },
                        'connections' : {
                            'CON_FMD_FSQL' : '372237f9-709a-48f8-8fb2-ce06940c990e',
                            'CON_FMD_API_FABRIC' : '644f217a-8761-4f19-b1b7-a40bfe0ec2af',
                            'CON_FMD_ASQL_01' : 'cf673e6a-13f6-4ebb-9cbb-4ba4ab390818',
                            'CON_FMD_ASQL_02' : '11a8e5fe-fbca-4822-9ba4-9162cf56e6dd',
                            'CON_FMD_ADLS_01' : 'a0581b6e-5e38-46eb-bab2-7f08e9a35c30',
                            'CON_FMD_FABRICPIPELINES' : '6d8146c6-a438-47df-94e2-540c552eb6d7',
                            'CON_FMD_ADF_PIPELINES' : '02e107b8-e97e-4b00-a28c-668cf9ce3d9a'
                        }
                    }
                ]



StatementMeta(, 5bcbb833-405a-41d2-a0ff-8103d99b7a53, 119, Finished, Available, Finished)

In [297]:
from json import loads, dumps
import json
import requests
import base64
import time
import uuid

from typing import Callable, List, Dict, Optional, Any
from datetime import datetime
from time import sleep, time
from dataclasses import dataclass, field

StatementMeta(, 5bcbb833-405a-41d2-a0ff-8103d99b7a53, 120, Finished, Available, Finished)

In [298]:
%run NB_FMD_UTILS

StatementMeta(, 5bcbb833-405a-41d2-a0ff-8103d99b7a53, 124, Finished, Available, Finished)

In [299]:
tasks=[]

StatementMeta(, 5bcbb833-405a-41d2-a0ff-8103d99b7a53, 125, Finished, Available, Finished)

In [300]:
fmd_api_access_token =  notebookutils.credentials.getToken('https://analysis.windows.net/powerbi/api')
fabric_session = create_fabric_session(fabric_token = fmd_api_access_token)

StatementMeta(, 5bcbb833-405a-41d2-a0ff-8103d99b7a53, 126, Finished, Available, Finished)

In [301]:
# check if token is valid
for token in [fmd_api_access_token]:
    if not token:
        continue
    header, payload, signature = token.split('.')
    payload += '=' * (-len(payload) % 4)  # Add padding
    token_dict = loads(base64.urlsafe_b64decode(payload))
    directory_id = token_dict.get("tid")
    timest = token_dict.get("exp")
    expiry = (datetime.fromtimestamp(timest) - datetime.now()).total_seconds() // 60
    expiry_str = str(expiry) if expiry < 5 else str(expiry)
    print(F"token {directory_id} will expire in {expiry_str} minutes at\t{datetime.fromtimestamp(timest)} UTC")
print(F"Current time:\t\t\t\t\t\t\t\t\t{datetime.now().replace(microsecond=0)} UTC")

StatementMeta(, 5bcbb833-405a-41d2-a0ff-8103d99b7a53, 127, Finished, Available, Finished)

token c183ff6a-8ca1-4185-9378-af980978377c will expire in 19.0 minutes at	2025-01-29 16:06:48 UTC
Current time:									2025-01-29 15:47:14 UTC


In [302]:
# Create necessary workspaces 
start = time()

# List all workspaces
workspaces = get_fabric_workspaces(fabric_session)

for environment in environments:

    # Loop through the workspace names and get their IDs
    for workspace_name in [environment['workspaces']['data']['name'], environment['workspaces']['code']['name'], configuration['workspace']['name']]:
        if workspace_name == environment['workspaces']['data']['name']:
            workspace = environment['workspaces']['data']
        elif workspace_name == environment['workspaces']['code']['name']:
            workspace = environment['workspaces']['code']
        elif workspace_name == configuration['workspace']['name']:
            workspace = configuration['workspace']

        # Check if the displayName exists in the workspaces
        matching_workspaces = [workspace for workspace in workspaces.get('value') if workspace['displayName'] == workspace_name]
        if matching_workspaces:
            workspace['id'] = matching_workspaces[0]['id']
            print(f"Workspace '{workspace_name}' found. Workspace ID: {workspace['id']}")
            tasks.append({"task_name":f"create item {workspace_name} initially", "task_duration": int(time() - start), "status": "success"})
            assign_fabric_workspace_capacity(fabric_session, workspace['id'], workspace['capacity_id'])
            tasks.append({"task_name":f"Workspace '{workspace_name}' connected to capacity ID: {workspace['capacity_id']}", "task_duration": int(time() - start), "status": "success"})
            print(f"Workspace '{workspace_name}' connected to capacity ID: {workspace['capacity_id']}")   
        else:
            print(f"Workspace '{workspace_name}' not found. Creating new workspace...")
            workspace_created = fabric_request(fabric_session, F"workspaces/", 'POST', payload={"displayName": workspace_name}, payloadtype='json')
            workspace['id'] = workspace_created['id']
            tasks.append({"task_name":f"create item {workspace_name} initially", "task_duration": int(time() - start), "status": "success"})
            assign_fabric_workspace_capacity(fabric_session, workspace['id'], workspace['capacity_id'])
            tasks.append({"task_name":f"Workspace '{workspace_name}' connected to capacity ID: {workspace['capacity_id']}", "task_duration": int(time() - start), "status": "success"})

            print(f"Workspace '{workspace_name}' created with ID: {workspace['id']} and capacity_id: {workspace['capacity_id']}")

        # Check if roles exists or create them
        assign_fabric_workspace_roles(fabric_session, workspace['id'], workspace['roles'])

    # Print the workspace IDs
    print(f"Workspace ID for data workspace {environment['environment_name']}: {environment['workspaces']['data']['id']}")
    print(f"Workspace ID for code workspace {environment['environment_name']}: {environment['workspaces']['code']['id']}")


StatementMeta(, 5bcbb833-405a-41d2-a0ff-8103d99b7a53, 128, Finished, Available, Finished)

Workspace 'FMD_FRAMEWORK_DATA_MH2 (dvlm)' found. Workspace ID: c5f8ac02-00db-4f2b-a030-6965683abb7a
Capacity found (560C908C-072A-478C-939C-CFC4398D2FDB)
Capacity (560C908C-072A-478C-939C-CFC4398D2FDB) assigning to workspace (c5f8ac02-00db-4f2b-a030-6965683abb7a)
Request accepted: Dont Wait for result
Workspace 'FMD_FRAMEWORK_DATA_MH2 (dvlm)' connected to capacity ID: 560C908C-072A-478C-939C-CFC4398D2FDB
Skipping: sg-fabric-contributor for c5f8ac02-00db-4f2b-a030-6965683abb7a
Skipping: sg-fabric-admin for c5f8ac02-00db-4f2b-a030-6965683abb7a
Workspace 'FMD_FRAMEWORK_CODE_MH2 (dvlm)' found. Workspace ID: 6a568632-74d2-41b2-84ba-b30a4fb450f4
Capacity found (560C908C-072A-478C-939C-CFC4398D2FDB)
Capacity (560C908C-072A-478C-939C-CFC4398D2FDB) assigning to workspace (6a568632-74d2-41b2-84ba-b30a4fb450f4)
Request accepted: Dont Wait for result
Workspace 'FMD_FRAMEWORK_CODE_MH2 (dvlm)' connected to capacity ID: 560C908C-072A-478C-939C-CFC4398D2FDB
Skipping: sg-fabric-contributor for 6a568632

In [304]:
# Read deployment manifest
deployment_manifest = {}
with open(f"{notebookutils.fs.getMountPath('/default')}/Files/{deployment_file}") as f:
    deployment_manifest = json.load(f)

StatementMeta(, 5bcbb833-405a-41d2-a0ff-8103d99b7a53, 130, Finished, Available, Finished)

In [305]:
# create empty items if not exists

for environment in environments:
    environment['guids_to_replace'] = []
    
    target_items = get_fabric_items(fabric_session, environment['workspaces']['code']['id'])

    for deployment_item in deployment_manifest['items']:
        # Check if target_items['value'] is empty
        if not target_items['value']:
            print(f"Creating: {deployment_item['displayName']}")
            item = deployment_item.copy()
            item.pop('definition')
            return_item = fabric_request(fabric_session, url=f"workspaces/{environment['workspaces']['code']['id']}/items/", method="POST", payload=item, payloadtype='json')
            deployment_item["new_id"] = return_item['id']
        else:
            for target_item in target_items['value']:
                if target_item['displayName'] == deployment_item['displayName']:
                    break
            else:
                print(f"Creating: {deployment_item['displayName']}")
                item = deployment_item.copy()
                item.pop('definition')
                return_item = fabric_request(fabric_session, url=f"workspaces/{environment['workspaces']['code']['id']}/items/", method="POST", payload=item, payloadtype='json')
                deployment_item["new_id"] = return_item['id']
                continue

            print(f"Skip existing: {deployment_item['displayName']}")
            deployment_item["new_id"] = target_item['id']

        environment['guids_to_replace'].append({"old_id": deployment_item["org_id"], "new_id": deployment_item["new_id"]})

    target_items = get_fabric_items(fabric_session, environment['workspaces']['data']['id'])

    for deployment_data in deployment_manifest['data']:
        for target_item in target_items['value']:
            if target_item['displayName'] == deployment_data['displayName']:
                break
        else:
            print(f"Creating: {deployment_data['displayName']}")
            return_item = fabric_request(fabric_session, url=f"workspaces/{environment['workspaces']['data']['id']}/items/", method="POST", payload=deployment_data, payloadtype='json')
            deployment_data["new_id"] = return_item['id']
            continue

        print(f"Skip existing: {deployment_data['displayName']}")
        deployment_data["new_id"] = target_item['id']

        environment['guids_to_replace'].append({"old_id": deployment_data["org_id"], "new_id": deployment_data["new_id"]})

    # skip because sqldatabase isn't supported yet
    # target_items = get_fabric_items(fabric_session, configuration['workspace']['id'])

    # for deployment_config in deployment_manifest['configuration']['items']:
    #     for target_item in target_items['value']:
    #         if target_item['displayName'] == deployment_config['displayName']:
    #             break
    #     else:
    #         print(f"Creating: {deployment_config['displayName']}")
    #         return_item = fabric_request(fabric_session, url=f"workspaces/{configuration['workspace']['id']}/items/", method="POST", payload=deployment_config, payloadtype='json')
    #         deployment_config["new_id"] = return_item['id']
    #         continue

    #     print(f"Skip existing: {deployment_config['displayName']}")
    #     deployment_config["new_id"] = target_item['id']

    #     environment['guids_to_replace'].append({"old_id": deployment_config["org_id"], "new_id": deployment_config["new_id"]})

StatementMeta(, 5bcbb833-405a-41d2-a0ff-8103d99b7a53, 131, Finished, Available, Finished)

Skip existing: NB_FMD_LOAD_BRONZE_SILVER
Skip existing: NB_FMD_LOAD_LANDING_BRONZE
Skip existing: PL_FMD_LOAD_LANDINGZONE
Skip existing: PL_FMD_LOAD_ALL
Skip existing: PL_FMD_LOAD_BRONZE
Skip existing: PL_FMD_LOAD_SILVER
Skip existing: PL_FMD_LDZ_COPY_FROM_ASQL_01
Skip existing: PL_FMD_LDZ_COPY_FROM_ONELAKE_TABLES_01
Skip existing: PL_FMD_LDZ_COPY_FROM_ADLS_01
Skip existing: PL_FMD_LDZ_COPY_FROM_ASQL_02
Skip existing: PL_FMD_LDZ_COMMAND_ASQL
Skip existing: PL_FMD_LDZ_COMMAND_ADLS
Skip existing: PL_FMD_LDZ_COMMAND_ONELAKE
Skip existing: PL_FMD_LDZ_COPY_FROM_ADF
Skip existing: PL_FMD_LDZ_COMMAND_ADF
Skip existing: NB_FMD_DQ_CLEANSING
Skip existing: LH_LANDINGZONE
Skip existing: LH_BRONZE_LAYER
Skip existing: LH_SILVER_LAYER
Skip existing: ENV_FMD
Skip existing: NB_FMD_LOAD_BRONZE_SILVER
Skip existing: NB_FMD_LOAD_LANDING_BRONZE
Skip existing: PL_FMD_LOAD_LANDINGZONE
Skip existing: PL_FMD_LOAD_ALL
Skip existing: PL_FMD_LOAD_BRONZE
Skip existing: PL_FMD_LOAD_SILVER
Skip existing: PL_FMD_LD

In [308]:
for environment in environments:
    # Append the remaining pairs
    environment['guids_to_replace'].append({"old_id": deployment_manifest["workspaces"]["workspace_data"], "new_id": environment['workspaces']['data']['id']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["workspaces"]["workspace_code"], "new_id": environment['workspaces']['code']['id']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["configuration"]["artifact_sql_framework"], "new_id": artifact_sql_framework})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_FSQL"], "new_id": environment['connections']['CON_FMD_FSQL']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_API_FABRIC"], "new_id": environment['connections']['CON_FMD_API_FABRIC']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_ASQL_01"], "new_id": environment['connections']['CON_FMD_ASQL_01']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_ASQL_02"], "new_id": environment['connections']['CON_FMD_ASQL_02']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_ADLS_01"], "new_id": environment['connections']['CON_FMD_ADLS_01']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_FABRICPIPELINES"], "new_id": environment['connections']['CON_FMD_FABRICPIPELINES']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["connections"]["CON_FMD_ADF_PIPELINES"], "new_id": environment['connections']['CON_FMD_ADF_PIPELINES']})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["logging"]["log_workspace_guid"], "new_id": logging["log_workspace_guid"]})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["logging"]["log_warehouse_guid"], "new_id": logging["log_warehouse_guid"]})
    environment['guids_to_replace'].append({"old_id": deployment_manifest["logging"]["log_warehouse_endpoint"], "new_id": logging["log_warehouse_endpoint"]})

StatementMeta(, 5bcbb833-405a-41d2-a0ff-8103d99b7a53, 134, Finished, Available, Finished)

In [309]:
items_to_deploy = deployment_manifest["items"]

for environment in environments:
    print(f"--------------------------")
    print(f"Processing: {environment['environment_name']}")
    
    existing_items_code = fabric_request(fabric_session, url=f"workspaces/{environment['workspaces']['code']['id']}/items/", method="GET")
    deploy_items(items_to_deploy, environment['guids_to_replace'], fmd_api_access_token, environment['workspaces']['code']['id'], existing_items_code["value"])

StatementMeta(, 5bcbb833-405a-41d2-a0ff-8103d99b7a53, 135, Finished, Available, Finished)

--------------------------
Processing: development
Deploy NB_FMD_LOAD_BRONZE_SILVER (Notebook)
Update 9ee9c740-7189-4aee-8499-dc423c3776aa
Deploy NB_FMD_LOAD_LANDING_BRONZE (Notebook)
Update b7f61c44-6a24-476b-a9b3-59ca4c9c7ad4
Deploy PL_FMD_LOAD_LANDINGZONE (DataPipeline)
Update 23417ffa-315a-4616-8912-26c954151d67
Deploy PL_FMD_LOAD_ALL (DataPipeline)
Update 61fca325-0ae4-497b-a01c-9a0565f05018
Deploy PL_FMD_LOAD_BRONZE (DataPipeline)
Update c7c2c6fc-71ae-4aa2-b253-b61c71eb0f1a
Deploy PL_FMD_LOAD_SILVER (DataPipeline)
Update 4ab242e2-174a-4872-8a0b-73b4ff0e1026
Deploy PL_FMD_LDZ_COPY_FROM_ASQL_01 (DataPipeline)
Update c171789d-4bda-4b76-adf4-66631e837cdd
Deploy PL_FMD_LDZ_COPY_FROM_ONELAKE_TABLES_01 (DataPipeline)
Update 20df98d3-a63d-43bb-abe7-a1a70157c9bf
Deploy PL_FMD_LDZ_COPY_FROM_ADLS_01 (DataPipeline)
Update a7d8acae-2116-430b-a59f-5d7fbb3be431
Deploy PL_FMD_LDZ_COPY_FROM_ASQL_02 (DataPipeline)
Update c91d03bc-a433-4b18-9a6b-0782055764c2
Deploy PL_FMD_LDZ_COMMAND_ASQL (DataPipe

In [310]:
display(tasks)

StatementMeta(, 5bcbb833-405a-41d2-a0ff-8103d99b7a53, 136, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, dc2e0c7e-dae6-4783-b82b-d669d6ab5eb7)